In [1]:
from joinboost_dp import *

In [2]:
sales = pd.read_csv("data/train.csv")

In [3]:
cons = dict()
for i in range(2):
    cons[i] = duckdb.connect(database=':memory:', check_same_thread=False)
    print("reading tables into duckdb...")
    cons[i].execute("CREATE OR REPLACE TABLE holidays AS SELECT * FROM 'data/holidays.csv';")
    cons[i].execute("CREATE OR REPLACE TABLE oil AS SELECT * FROM 'data/oil.csv';")
    cons[i].execute("CREATE OR REPLACE TABLE transactions AS SELECT * FROM 'data/transactions.csv';")
    cons[i].execute("CREATE OR REPLACE TABLE stores AS SELECT * FROM 'data/stores.csv';")
    cons[i].execute("CREATE OR REPLACE TABLE items AS SELECT * FROM 'data/items.csv';")
#     cons[i].execute("CREATE OR REPLACE TABLE test AS SELECT * FROM 'data/test.csv';")
    print("done")

reading tables into duckdb...
done
reading tables into duckdb...
done


In [4]:
jg = joinGraph("favorita", cons, log=False, max_leaves = 8,  learning_rate=0.1, target_variable ="Y", fact_pandas=True, fact_pandas_join=True)
jg.add_table("sales", [], [], fact=True, fact_table=sales)
jg.add_table("holidays", ["htype", "locale", "locale_name", "transferred","f2"], [2,2,2,2,2])
jg.add_table("oil", ["dcoilwtico","f3"], [2,2])
jg.add_table("transactions", ["transactions","f5"], [2,2])
jg.add_table("stores", ["city","state","stype","cluster","f4"], [2,2,2,2,2])
jg.add_table("items", ["family","class","perishable","f1"], [2,2,2,2])
jg.join("sales", "items", ["item_nbr"], ["item_nbr"])
jg.join("sales", "transactions", ["tid"], ["tid"])
jg.join("transactions", "stores", ["store_nbr"], ["store_nbr"])
jg.join("transactions", "holidays", ["date"], ["date"])
jg.join("holidays", "oil", ["date"], ["date"])

In [7]:
jg.create_dummy_model(replace=True)
for i in cons:
    cons[i].register('sales', sales)
total = 0
for i in range(100):
    jg.create_base_node()
    jg.build_gradient_tree()
    jg.update_error()
    jg.clean_leaves()
    jg.clean_table()
    print(str(i) + ", " + str(jg.totaltime))
#     q =  jg.predict_succ("train")
#     cons[1].execute(q)
#     train_rmse = cons[1].fetchall()[0][0]
    
#     q = jg.predict_succ("test")
#     cons[1].execute(q)
#     test_rmse = cons[1].fetchall()[0][0]
    
#     print(str(i) + ", " + str(jg.totaltime) + ", " + str(train_rmse) + ", " + str(test_rmse))

0, 6.85822868347168
1, 13.369253635406494
2, 19.902899503707886
3, 26.342350959777832
4, 33.0956711769104
5, 39.4733202457428
6, 46.223867893218994
7, 52.933751344680786
8, 59.612147092819214
9, 66.25364971160889
10, 73.28335547447205
11, 80.1885347366333
12, 86.63545632362366
13, 93.27619552612305
14, 100.11036610603333
15, 107.15573644638062
16, 114.07650542259216
17, 120.9218807220459
18, 127.64583230018616
19, 135.03854870796204
20, 142.03516817092896
21, 148.4516727924347
22, 155.46517658233643
23, 161.60451340675354
24, 168.8393247127533
25, 175.32075762748718
26, 181.69782066345215
27, 187.5407314300537
28, 192.66916751861572
29, 199.00664401054382
30, 204.3261799812317
31, 210.20592164993286
32, 217.47933626174927
33, 223.14206624031067
34, 229.99482989311218
35, 237.5413863658905
36, 243.0766909122467
37, 249.9735381603241
38, 255.88164639472961
39, 262.2987287044525
40, 269.01419615745544
41, 275.8949761390686
42, 282.6517617702484
43, 288.34718585014343
44, 295.1724841594696